In [ ]:
from pathlib import Path
import re
import contractions
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp
import seaborn as sb
from sklearn.feature_extraction.text import CountVectorizer

## ACL-14-Short: Brief Description
Exclusively tweets.

The raw data is in a text file and spread over groups of 3 lines such that for $i \in [0,nlines/3]$:
* line i * 3 --> contains the context
* line i * 3 + 1 --> contains the sentiment target
* line i * 3 + 2 --> contains the sentiment polarity

## Load Data

In [ ]:
proj_path = Path('/home/ekarakoylu/PROJEX/directed_sentiment_nli')
acl_data_path = proj_path /'data/acl-14-short'
sentfin_dp = proj_path / 'data/sentfin'
newsmtsc_dp = proj_path /'data/newsmtsc'

In [ ]:
def load_data_2_dframe(path: Path | str):
    """
    This functions parses data files in the ACL-14-Short format. 
        Input: 
            [pathlib.Path or str] data path
        Output:
            DataFrame with columns 'context' (str), 'target'(str), 'polarity'(int).

    """
    try:
        with open(path) as f:
            flines = f.read().splitlines()
        fdict = dict(
            context=flines[::3],
            target=flines[1::3],
            polarity=flines[2::3])
        df = pd.DataFrame(fdict)
        df['polarity'] = pd.to_numeric(df.polarity)
        return df
    except FileNotFoundError as err:
        # logging statement?
        raise err
    

In [ ]:
df_train = load_data_2_dframe(acl_data_path/'raw/train.raw')
df_test = load_data_2_dframe(acl_data_path/'raw/test.raw')

In [ ]:
df_train.info()

## Cleaning:
Steps involved are:
1. Removing unnecessary characters (including spurious blank spaces)
2. Clean contractions
3. Replace emojis with their textual equivalent.

In [ ]:
def clean_step1(text):
    # Remove links
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)

    # Convert HTML references
    text = re.sub('&amp', 'and', text)
    text = re.sub('&lt', '<', text)
    text = re.sub('&gt', '>', text)

    # Remove new line characters
    text = re.sub('[\r\n]+', ' ', text)
    
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove multiple space characters
    text = re.sub('\s+',' ', text)
    
    # Convert to lowercase
    text = text.lower()
    return text

In [ ]:
def clean_step2(text):
    try:
        return contractions.fix(text)
    except:
        return text

In [ ]:
def clean_dataframe(df):
    dclean = pd.concat(
        (df.explode('opinions')['opinions'].apply(pd.Series), 
        df.drop('opinions', axis=1)),
        axis=1, join='outer').drop(0, axis=1).dropna().reset_index(drop=True)
    return dclean


### EDA
This section features EDA steps for the ACL-14 dataset. Specific steps are as follows
1. Most common bigrams and trigrams to detect possible spurious/problematic patterns
2. General sentiment (inferred using Vader and TextBlob) and comparisons to target polarities.
3. Class distributions

In [ ]:
def get_top_k_ngrams(corpus, k=None, ngram=2):
    vec = CountVectorizer(ngram_range=(ngram, ngram), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

    d_wf = pd.DataFrame(words_freq, columns = ['TweetText' , 'count'])
    d_wf.groupby('TweetText').sum()['count'].sort_values(ascending=False).iplot(
        kind='bar',
        yTitle='Count',
        linecolor='black',
        title='Top 20 bigrams in Tweet before removing spams')
    return words_freq[:k], d_wf

In [ ]:
#d_bigram = get_top_k_ngrams(df.processed_context, k=20)
#d_trigram = get_top_k_ngrams(df.processed_context, k=20, ngram=3)